In [1]:
## Import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts
import pickle

In [2]:
# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z

In [3]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
## Import functions from Qiskit
from qiskit                     import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.providers.aer       import QasmSimulator
from qiskit.tools.monitor       import job_monitor
from qiskit.circuit             import Parameter, ParameterVector
from qiskit.quantum_info        import Statevector, Pauli
from qiskit.opflow.state_fns    import CircuitStateFn
from qiskit.opflow.expectations import PauliExpectation
from qiskit.utils               import QuantumInstance
from qiskit.opflow              import PauliOp, SummedOp, CircuitSampler, StateFn

In [5]:
# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info                  import state_fidelity

In [ ]:
## Mitiq libraries
from mitiq import zne
from qiskit.result import Result
from qiskit.result.models import ExperimentResult
from qiskit.result.models import ExperimentResultData
from qiskit.result.models import QobjExperimentHeader

In [8]:
from itertools import chain

## Create the circuit

In [9]:
def Heisenberg_YBE_variational(num_qubits,p):

    circ  = QuantumCircuit(num_qubits)
    count = 0
    
    def XYZ_variational(circ,i,j,params):
        circ.cx(i,j)
        circ.rx(params[0],i)
        circ.rx(-np.pi/2,i)
        circ.h(i)
        circ.rz(params[1],j)

        circ.cx(i,j)
        circ.h(i)
        circ.rz(params[2],j)

        circ.cx(i,j)
        circ.rx(np.pi/2,i)
        circ.rx(-np.pi/2,j)

    circ.rx(np.pi,[1,2])
    
    XYZ_variational(circ,1,2,p[count:count+3])
    count += 3
    XYZ_variational(circ,0,1,p[count:count+3])
    count += 3
    XYZ_variational(circ,1,2,p[count:count+3])
    count += 3
    XYZ_variational(circ,0,1,p[count:count+3])
    count += 3
    XYZ_variational(circ,1,2,p[count:count+3])
    count += 3

    return circ

In [10]:
pvqd_opt_params = [0.6382017062070897,
0.5999999987484098,
0.6382017062066773,
3.0088034895496003,
-3.0869200336945677,
0.4709531470409451,
2.163149581322057,
3.480816125849344,
-2.0741264452466974,
1.2330206913091548,
3.1275100711382064,
1.593744340473751,
6.107319841483039,
3.0177717815840808,
-3.24901805128811]

In [11]:
## Create the circuit
# Define the final circuit that is used to compute the fidelity 
fqr = QuantumRegister(7)
fqc = QuantumCircuit(fqr)
#fqc.rx(np.pi, [3, 5]) # Cannot use X gate due to a bug in mitq, rx(pi) does the same thing
fqc.id([0, 1, 2, 4, 6]) # Need to put identities since mitq cannot handle unused qubits
fqc.append(Heisenberg_YBE_variational(3,pvqd_opt_params), [fqr[1], fqr[3], fqr[5]])

## Run on hardware

In [15]:
## Info for IBM
#IBMQ.save_account('MY_API_TOKEN')
#IBMQ.enable_account('MY_API_TOKEN')
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [16]:
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')
# Simulated backend based on ibmq_jakarta's device noise profile
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

In [17]:
shots = 8192
backend = sim_noisy_jakarta
#backend = jakarta

In [18]:
# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid

In [38]:
def zne_job(tomo_circs, backend, optimization_level, shots):

    # This function runs the tomography circuits and unrolls the gates to increase the noise level
    # The counts that are obtained for the differnt noise levels are then extrapolated to the zero-noise level

    zne_result_list = []
    scale_factors = [1.0, 2.0, 3.0]


    # Unfold the tomography circuit by a scale factor and evaluate them 
    noise_scaled_circuits = [[zne.scaling.fold_global(circ, s) for s in scale_factors] for circ in tomo_circs] 
    noise_scaled_circuits = list(chain(*noise_scaled_circuits)) 

    
    job    = execute(noise_scaled_circuits, backend=backend, optimization_level=optimization_level, shots=shots)
    result = job.result()
    job_id = job.job_id()
    ## Now we will save these results in a file, together with the circuits that have generated them
    
    pickle_file = "./hw_data/tomography_id_"+str(job_id)
    pickle_data = {}
    pickle_data["result"]          = result
    pickle_data["tomo_circs"]      = tomo_circs
    
    # Dump on file
    with open(pickle_file,'wb+') as f:
        pickle.dump(pickle_data, f)
    
    print("JOB "+str(job_id)+" SUCCESSFULLY RUN AND SAVED")
    return str(job_id), result

In [39]:
# Create the tomography circuits
st_qcs = state_tomography_circuits(fqc.decompose(), [fqr[1], fqr[3], fqr[5]])

In [40]:
# Repeat fidelity measurement
reps = 1 # Needs to be 8 in the final execution
result_list = []
job_id_list = []
for count in range(reps):
    print("\n REPETITION "+str(count+1)+"\n")
    
    zne_job_id, zne_res = zne_job(st_qcs, backend=backend, optimization_level=0, shots=shots)
    result_list.append(zne_res)
    job_id_list.append(zne_job_id)


 REPETITION 1

JOB a24723dc-6ef2-48aa-abc0-c7f984282fe8 SUCCESSFULLY RUN AND SAVED


## Retrieve the results

#### From result list of this notebook

In [22]:
def zne_results_from_notebook(result,tomo_circs, zne_order, shots):

    
    count_list = result.get_counts()
    ordered_bitstrings = dict(sorted(count_list[0].items()))
    
    zne_result_list = []
    scale_factors = [1.0, 2.0, 3.0]

    for i in range(len(tomo_circs)):
        counts_dict = {}

        # Loop over the results of the scaled circuits and collect the data in the correct form
        for key in ordered_bitstrings.keys():
            counts_list_zne = []
            for count in count_list[i*len(scale_factors):len(scale_factors)*(i+1)]:
                counts_list_zne.append(count[key])
            # Here we extrapolate the counts to zero noise and round to the closest integer 
            zne_counts_value = int(zne.PolyFactory.extrapolate(scale_factors, counts_list_zne, order=zne_order)) 
            if zne_counts_value < 0:
                zne_counts_value = 0
            counts_dict[key] = zne_counts_value
        zne_result_list.append(counts_dict)
        
    # To work with the StateTomographyFitter we need to put the result into a Qiskit Result() object
    name_list = [circ.name for circ in tomo_circs]
    results_tmp = [[ExperimentResult(shots=shots, success=True, data=ExperimentResultData(counts=result_i), header=QobjExperimentHeader(name=name_i))] for (name_i, result_i) in zip(name_list, zne_result_list)]
    results = [Result(backend_name="zne", backend_version="zne", qobj_id='0', job_id='0', success=True, results=result_i) for result_i in results_tmp]

    return results 

In [23]:
fids     = []
for result in result_list:
    #print(job)
    zne_res  = zne_results_from_notebook(result=result,tomo_circs=st_qcs, zne_order=2, shots=shots)
    fids.append(state_tomo(zne_res, st_qcs))

In [24]:
## Print the final result
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fids), np.std(fids))) 

state tomography fidelity = 0.8734 ± 0.0000


#### From data file

In [27]:
def zne_results_from_file(filename, zne_order, shots):

    
    result_file = pickle.load(open(filename,'rb'))
    print("\nLOADED FILE: "+filename)
    
    tomo_circs = result_file["tomo_circs"]
    result = result_file["result"]
    
    count_list = result.get_counts()
    ordered_bitstrings = dict(sorted(count_list[0].items()))
    
    zne_result_list = []
    scale_factors = [1.0, 2.0, 3.0]

    for i in range(len(tomo_circs)):
        counts_dict = {}

        # Loop over the results of the scaled circuits and collect the data in the correct form
        for key in ordered_bitstrings.keys():
            counts_list_zne = []
            for count in count_list[i*len(scale_factors):len(scale_factors)*(i+1)]:
                counts_list_zne.append(count[key])
            # Here we extrapolate the counts to zero noise and round to the closest integer 
            zne_counts_value = int(zne.PolyFactory.extrapolate(scale_factors, counts_list_zne, order=zne_order)) 
            if zne_counts_value < 0:
                zne_counts_value = 0
            counts_dict[key] = zne_counts_value
        zne_result_list.append(counts_dict)
        
    # To work with the StateTomographyFitter we need to put the result into a Qiskit Result() object
    name_list = [circ.name for circ in tomo_circs]
    results_tmp = [[ExperimentResult(shots=shots, success=True, data=ExperimentResultData(counts=result_i), header=QobjExperimentHeader(name=name_i))] for (name_i, result_i) in zip(name_list, zne_result_list)]
    results = [Result(backend_name="zne", backend_version="zne", qobj_id='0', job_id='0', success=True, results=result_i) for result_i in results_tmp]

    return results 

In [33]:
# the list of job to evaluate
file_fids     = []

for job in job_id_list:
    #print(job)
    
    job_file = "./hw_data/tomography_id_"+job
    zne_res  = zne_results_from_file(filename=job_file, zne_order=2, shots=shots)
    file_fids.append(state_tomo(zne_res, st_qcs))


LOADED FILE: ./hw_data/tomography_id_281000ce-5d86-4ab9-a83d-7255b21799f1


In [34]:
## Print the final result
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(file_fids), np.std(file_fids))) 

state tomography fidelity = 0.8734 ± 0.0000


#### From IBM Quantum

In [35]:
def zne_results_from_IBMQ(backend,jobid,tomo_circs, zne_order, shots):
    
    job    = backend.retrieve_job('JobID') # Maybe a [0] is needed if it doesn't work properly
    result = job.result()
    
    count_list = result.get_counts()
    ordered_bitstrings = dict(sorted(count_list[0].items()))
    
    zne_result_list = []
    scale_factors = [1.0, 2.0, 3.0]

    for i in range(len(tomo_circs)):
        counts_dict = {}

        # Loop over the results of the scaled circuits and collect the data in the correct form
        for key in ordered_bitstrings.keys():
            counts_list_zne = []
            for count in count_list[i*len(scale_factors):len(scale_factors)*(i+1)]:
                counts_list_zne.append(count[key])
            # Here we extrapolate the counts to zero noise and round to the closest integer 
            zne_counts_value = int(zne.PolyFactory.extrapolate(scale_factors, counts_list_zne, order=zne_order)) 
            if zne_counts_value < 0:
                zne_counts_value = 0
            counts_dict[key] = zne_counts_value
        zne_result_list.append(counts_dict)
        
    # To work with the StateTomographyFitter we need to put the result into a Qiskit Result() object
    name_list = [circ.name for circ in tomo_circs]
    results_tmp = [[ExperimentResult(shots=shots, success=True, data=ExperimentResultData(counts=result_i), header=QobjExperimentHeader(name=name_i))] for (name_i, result_i) in zip(name_list, zne_result_list)]
    results = [Result(backend_name="zne", backend_version="zne", qobj_id='0', job_id='0', success=True, results=result_i) for result_i in results_tmp]

    return results

In [36]:
# the list of job to evaluate
st_qcs   = state_tomography_circuits(fqc.decompose(), [fqr[1], fqr[3], fqr[5]])
#backend = jakarta
ibmq_fids     = []

for job in job_id_list:
    #print(job)
    
    job_file = "./hw_data/tomography_id_"+job
    zne_res  = zne_results_from_IBMQ(backend=backend,jobid=job, zne_order=2, shots=shots)
    ibmq_fids.append(state_tomo(zne_res, st_qcs))

TypeError: zne_results_from_IBMQ() missing 1 required positional argument: 'tomo_circs'

In [32]:
## Print the final result
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(ibmq_fids), np.std(ibmq_fids))) 

state tomography fidelity = nan ± nan
